In [7]:
# Setup Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Global imports
import json
import pandas as pd
import seaborn as sns

# Typing import

from typing import List, Tuple

# Specific imports
from rich import print
from termcolor import cprint
from time import time
from typing import Dict, Tuple, List

sns.set_theme('notebook')
sns.set_style("whitegrid")
sns.set_context("paper")
sns.color_palette("hls", 8)

def print_bl():
    print("\n")

def print_red(*args):
    for arg in args:
        cprint(arg, "red", end=' ')  # Using end=' ' to print all arguments on the same line

def print_green(*args):
    for arg in args:
        cprint(arg, "green", end=' ')  # Using end=' ' to print all arguments on the same line

def print_highlight(*args):
    for arg in args:
        cprint(arg, "magenta", "on_white", end=' ')  # Using end=' ' to print all arguments on the same line

def print_blue(*args):
    for arg in args:
        cprint(arg, "blue", end=' ')  # Using end=' ' to print all arguments on the same line

In [8]:
class GroupsLoader:
    """
    Class to load the groups from the json files.

    Attributes:
        groups_path (str): The path to the groups json files.
        dataset_index (int): The dataset index.
        groups (List[pd.DataFrame]): The list of groups dataframes.
        ego_vehicles (List[int]): The list of ego vehicle IDs.

    Methods:
        load_groups: Load the groups from the json files.
    """

    def __init__(self, groups_path: str, dataset_index: int = 1):
        self.groups_path = groups_path
        self.dataset_index = dataset_index
        self.groups = []
        self.ego_vehicles = []

    def load_groups(self) -> Tuple[List[pd.DataFrame], List[int]]:
        """
        Load the groups from the json files.

        Args:
            None

        Returns:
            A tuple containing the groups and the ego vehicles.
        
        Raises:
            FileNotFoundError: If the groups or ego vehicles file does not exist.
            ValueError: If the JSON content is invalid or cannot be processed.
        """
        try:
            with open(f"{self.groups_path}/groups_{self.dataset_index}.json", 'r') as f:
                data = json.load(f)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Groups file not found: {e.filename}")
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON from groups file: {e.msg}")

        try:
            self.groups = [pd.read_json(data[f]) for f in data.keys()]
        except ValueError as e:
            raise ValueError(f"Error reading JSON data into DataFrame: {e}")

        try:
            with open(f"{self.groups_path}/ego_vehicles_{self.dataset_index}.json", 'r') as f:
                self.ego_vehicles = json.load(f)
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Ego vehicles file not found: {e.filename}")
        except json.JSONDecodeError as e:
            raise ValueError(f"Error decoding JSON from ego vehicles file: {e}")

        return self.groups, self.ego_vehicles

In [28]:
class RounDPromptPopulator:
    """
    Class to populate the prompts.

    Takes the groups and ego vehicles and populates the prompt template. Saves the prompts in a JSON array.

    Attributes:
        groups_location (str): The path to the groups JSON files.
        template_path (str): The path to the template files.
        dataset_index (int): The dataset index.
        prompts (List[str]): The list of populated prompt strings.
    """

    def __init__(self, groups_location: str, template_path: str, dataset_index: int = 1):
        self.groups_location = groups_location
        self.template_path = template_path
        self.dataset_index = dataset_index
        self.prompts = []

        ### Error handling for loading groups and templates
        try:
            self.groups_loader = GroupsLoader(self.groups_location, self.dataset_index)
            self.groups, self.ego_vehicles = self.groups_loader.load_groups()
        except Exception as e:
            raise RuntimeError(f"Failed to load groups and ego vehicles: {e}")
        try:
            self.instructions_template, self.task_template, self.role_template, self.answer_template = self.load_templates()
        except Exception as e:
            raise RuntimeError(f"Failed to load templates: {e}")

    def load_templates(self) -> List[str]:
        """
        Load the prompt templates.

        Returns:
            A list containing the instructions, task, and role templates.
        
        Raises:
            FileNotFoundError: If a template file is not found.
            IOError: If there is an error reading a template file.
        """
        ### Error handling for loading templates
        try:
            with open(f"{self.template_path}/instructions_template.txt", 'r') as f:
                instructions_template = f.read()
            with open(f"{self.template_path}/task_template.txt", 'r') as f:
                task_template = f.read()
            with open(f"{self.template_path}/role_template.txt", 'r') as f:
                role_template = f.read()
            with open(f"{self.template_path}/answer_template.txt", 'r') as f:
                answer_template = f.read()
        except FileNotFoundError as e:
            raise FileNotFoundError(f"Template file not found: {e.filename}")
        except IOError as e:
            raise IOError(f"Error reading template file: {e}")

        return instructions_template, task_template, role_template, answer_template
    
    def relative_group(self, group_index: int, frame_tame_diff_ms: int = 400) -> pd.DataFrame:
        """
        Transforms frame numbering into temporal relative frame numbering.

        Args:
            group_index (int): The index of the group to transform.
            frame_tame_diff_ms (int): The time difference between frames in milliseconds.

        Returns:
            A DataFrame containing the transformed group.
        
        Raises:
            IndexError: If the group_index is out of range.
            KeyError: If expected keys are not found in the DataFrame.
        """
        ### Error handling for group index
        if group_index >= len(self.groups):
            raise IndexError(f"Group index {group_index} out of range.")

        group = self.groups[group_index]

        group['frame'] = (-group['frame'].max() + group['frame']) * frame_tame_diff_ms / 1000

        return group
    
    def row2str(self, row: pd.Series) -> str:
        """
        Convert a row to a string.

        Args:
            row (pd.Series): The row to convert.

        Returns:
            A string containing the row information.
        
        Raises:
            KeyError: If expected keys are not found in the row.
        """
        info = (
            f"At t={row['frame']} s, vehicle with id {row['trackId']} is at position ({row['xCenter_new']:.2f}, {row['yCenter_new']:.2f}) with heading {row['heading']:.2f} degrees."
            f"It's x axis speed is {row['xVelocity_new']:.2f} m/s and y axis speed is {row['yVelocity_new']:.2f} m/s. It's x axis cceleration is {row['xAcceleration_new']:.2f} m/s^2 "
            f"and y axis acceleration is {row['yAcceleration_new']:.2f} m/s^2. The width of the vehicle is {row['width']:.2f} m and its length is "
            f"{row['length']:.2f} m."
        )

        return info
    
    def get_prompt_static_info(self, group_index: int = 0) -> str:
        """
        Get the static information for the prompt.

        Args:
            group_index (int): The index of the group to get the static information.

        Returns:
            A string containing the static information for the prompt.
        
        Raises:
            KeyError: If expected keys are not found in the DataFrame.
        """
        group = self.relative_group(group_index)
        ego_vehicle_id = list(self.ego_vehicles.keys())[group_index]

        vehicle_ids = ", ".join([str(id) for id in group['trackId'].unique()])
        info = f"Vehicles present in the group have ids: {vehicle_ids}. "
        info += f"The ego vehicle is vehicle with id {ego_vehicle_id}."

        return info

    def populate_prompt(self, group_index: int = 0) -> List[Dict[str, str]]:
        """
        Populate the prompts. Takes a template and a vehicle group and returns a list of dictionaries with populated prompts.

        Args:
            group_index (int): The index of the group to populate.

        Returns:
            list: A list containing dictionaries with roles 'system' and 'user' and their corresponding content.
        
        Raises:
            ValueError: If the group DataFrame is empty.
        """
        if group_index >= len(self.groups):
            raise IndexError(f"Group index {group_index} out of range.")
        
        group = self.relative_group(group_index)
        
        if group.empty:
            raise ValueError(f"No data found for group index {group_index}.")
            
        frame_groups = group.groupby('frame')

        # Generate system prompt
        system_prompt = f"{self.role_template}\n\n{self.instructions_template}\n\n{self.answer_template}\n\n"

        # Generate user prompt
        user_prompt = (
            f"{self.get_prompt_static_info(group_index)}\n\n"
            "The information for each vehicle is as follows:\n"
        )
        
        for frame, frame_group in frame_groups:
            for index, row in frame_group.iterrows():
                user_prompt += f"{self.row2str(row)}\n"

        # Add task information
        user_prompt += f"\n{self.task_template}"

        message = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        
        return message
    
    def save_prompt_to_file(self, group_index: int = 0, filename: str = 'prompt.txt'):
        """
        Save the populated prompt to a file. Only for testing purposes. Main saving function is NOT this one.

        Args:
            group_index (int): The index of the group to populate.
            filename (str): The name of the file to save the prompt.
        
        Raises:
            IOError: If there is an error writing to the file.
        """
        ### Error handling for group index
        if group_index >= len(self.groups):
            raise IndexError(f"Group index {group_index} out of range.")
        
        ### Error handling for writing to file
        try:
            prompt = self.populate_prompt(group_index)
            print(f"Saving prompt to file {filename}...")
            print(f"System prompt:\n{prompt[0]['content']}")
            print(f"User prompt:\n{prompt[1]['content']}")

            with open(filename, 'w') as file:
                for message in prompt:
                    file.write(f"{message['content']}\n")
        except IOError as e:
            raise IOError(f"Error writing to file {filename}: {e}")
        
    def save_top_20_longest_prompts(self, filename: str = 'top_20_longest_prompts.json'):
        """
        Generate all prompts, find the 20 longest, and save them to a file.

        Args:
            filename (str): The name of the file to save the longest prompts.
        
        Raises:
            IOError: If there is an error writing to the file.
        """
        all_prompts = []

        for group_index in range(len(self.groups)):
            try:
                prompt = self.populate_prompt(group_index)
                combined_prompt = f"{prompt[0]['content']}\n{prompt[1]['content']}"
                all_prompts.append({
                    'group_index': group_index,
                    'length': len(combined_prompt),
                    'prompt': prompt
                })
            except Exception as e:
                print(f"Error generating prompt for group {group_index}: {e}")

        # Sort prompts by length in descending order and select the top 20
        top_20_prompts = sorted(all_prompts, key=lambda x: x['length'], reverse=True)[:20]

        # Extract the relevant information for saving
        top_20_prompts_content = [
            {
                'group_index': item['group_index'],
                'prompt': item['prompt']
            }
            for item in top_20_prompts
        ]

        # Save the top 20 longest prompts to a JSON file
        try:
            with open(filename, 'w') as file:
                json.dump(top_20_prompts_content, file, indent=4)
        except IOError as e:
            raise IOError(f"Error writing to file {filename}: {e}")

    def generate_and_save_all_prompts(self, folder: str = None):
        """
        Generate messages with system and user content for all groups and save them to a JSON file.

        Args:
            filename (str): Folder where the prompts will be saved.
        
        Raises:
            IOError: If there is an error writing to the file.
        """
        all_prompts = []

        for group_index in range(len(self.groups)):
            ### Error handling for generating prompt
            try:
                prompt = self.populate_prompt(group_index)
                #combine user and system prompts
                prompt = f"{prompt[0]['content']}\n{prompt[1]['content']}"
                all_prompts.append(prompt)
            except Exception as e:
                print(f"Error generating prompt for group {group_index}: {e}")
                

        filename = folder + '/' + f"prompts_{self.dataset_index}.json"
        try:
            with open(filename, 'w') as file:
                json.dump(all_prompts, file, indent=4)
        except IOError as e:
            raise IOError(f"Error writing to file {filename}: {e}")


In [29]:
groups_location = "../../data/rounD/groups_400"
prompts_destination = "../../data/rounD/groups_400"
template_path = "./templates/rounD_templates"

prompt_populator = RounDPromptPopulator(groups_location=groups_location, template_path=template_path, dataset_index=3)
prompt_populator.groups_loader.groups[0].columns


Index(['recordingId', 'trackId', 'frame', 'trackLifetime', 'xCenter',
       'yCenter', 'heading', 'width', 'length', 'xVelocity', 'yVelocity',
       'xAcceleration', 'yAcceleration', 'lonVelocity', 'latVelocity',
       'lonAcceleration', 'latAcceleration', 'xCenter_new', 'yCenter_new',
       'xVelocity_new', 'yVelocity_new', 'xAcceleration_new',
       'yAcceleration_new'],
      dtype='object')

In [30]:
prompt_populator.save_prompt_to_file(group_index=1, filename='prompt.txt')

Saving prompt to file prompt.txt...

System prompt:
Role:
        - You are a data scientist specialized in Autonomous Driving.
        - You hold deep and extensive knowledge of the relationship between traffic flow and the data that it 
generates.
        - You have considerable experience in driving, and as such, you can evaluate the behavior of a vehicle and 
its driver based solely on positional data, both current and historical.
        - You have intimate knowledge about dense traffic maneuvers, such as overtakes, merging into and out of 
lanes, intersection traversal, and particularly, roundabout navigation.
        - You possess a deep understanding of the various datasets available to the Autonomous Driving community, 
particularly the highD, exiD and rounD datasets.
        - You can reason out loud and provide in depth explanations of your chain of thought.
        - You provide meaningful and extensive answers, in which you explain everything that is required, as well 
as include any information you deem necessary to further support your explanation.
    - Your reasoning can incorporate all the information that is given in this prompt, and you are able to 
differentiate between past and present information when it is properly labeled.
        - Be decisive. Do not use conditionals when explaining the information. Convey uncertainty in a reasonable 
manner.

Instructions:
        - The data that is used as input has a temporal spacing of 0.4 seconds. The data has been gathered from the
first appearance of the ego vehicle to the instant right before it enters the roundabout.
        - The information that you are being provided relates to every vehicle that is present in the roundabout 
and the surrounding environment.
        - The coordinate system is as follows:
                - All positions are relative to the center of the roundabout.
                - The x axis is roughly aligned with the East-West axis, and grows towards the East.
                - The y axis is roughly aligned with the North-South axis, and grows towards the North.
                - A vehicle's heading is positive in the counterclockwise direction.
        - Road information:
                - The geometry of the roundabout is as follows:
                        - The center is located at (0, 0).
                        - The inner radius of the roundabout is 15.75 meters. The outer radius is 25.30 meters.
                        - The roundabout has four entry spots, located at: 
                                - north: (1.17, 28.48) m
                                - south: (-0.66, -28.45) m
                                - east: (28.75, 3.33) m
                                - west: (-27.33, -4.39) m
                        - The roundabout has four exit spots, located at: 
                                - north: (10.20, 26.28) m
                                - south: (-9.41, -26.53) m
                                - east: (27.97, -5.95) m
                                - west: (-27.31, 4.53) m
                - The roundabout has two lanes. Every entry spot has two lanes, while the exit spots have one lane 
each.
                - Vehicles are required to leave the roundabout from the exit lane, but due to traffic conditions 
and driver relaxation, that might not always be the case.
                - Vehicles are required to observe a 40 km/h (11,1 m/s) speed limit around the roundabout. This is 
not always the case.   

Answer template:
    - The answer needs to have the following content:
        - Provide an analysis of the situation. DO NOT list every value of information you have been provided, just
provide a qualitative analysis of the situation and use data whenever you need to support your reasoning.
        - Choose an action pair:
                - The first action, chosen from LIST 1, pertains to the speed of the ego vehicle.
                - The second action, chosen from LIST 2, pertains to the roundabout lane that the ego vehicle will 
choose.
   

User prompt:
Vehicles present in the group have ids: 7, 8, 9, 10, 11, 12, 13. The ego vehicle is vehicle with id 10.

The information for each vehicle is as follows:
At t=-0.8 s, vehicle with id 7.0 is at position (-52.87, -7.24) with heading 337.39 degrees.It's x axis speed is 
12.72 m/s and y axis speed is -2.26 m/s. It's x axis cceleration is 1.49 m/s^2 and y axis acceleration is -1.76 
m/s^2. The width of the vehicle is 2.08 m and its length is 4.49 m.
At t=-0.8 s, vehicle with id 8.0 is at position (30.71, -6.28) with heading 359.20 degrees.It's x axis speed is 
10.31 m/s and y axis speed is 2.12 m/s. It's x axis cceleration is 2.85 m/s^2 and y axis acceleration is -1.59 
m/s^2. The width of the vehicle is 1.94 m and its length is 5.00 m.
At t=-0.8 s, vehicle with id 9.0 is at position (24.78, 2.71) with heading 150.41 degrees.It's x axis speed is 
-5.67 m/s and y axis speed is 2.13 m/s. It's x axis cceleration is -0.55 m/s^2 and y axis acceleration is 2.36 
m/s^2. The width of the vehicle is 1.94 m and its length is 3.53 m.
At t=-0.8 s, vehicle with id 10.0 is at position (42.48, 3.60) with heading 159.56 degrees.It's x axis speed is 
-4.91 m/s and y axis speed is 0.56 m/s. It's x axis cceleration is -0.34 m/s^2 and y axis acceleration is 0.15 
m/s^2. The width of the vehicle is 2.06 m and its length is 4.94 m.
At t=-0.8 s, vehicle with id 11.0 is at position (29.54, 5.24) with heading 154.98 degrees.It's x axis speed is 
-3.95 m/s and y axis speed is 0.97 m/s. It's x axis cceleration is -1.19 m/s^2 and y axis acceleration is 1.48 
m/s^2. The width of the vehicle is 2.00 m and its length is 4.19 m.
At t=-0.8 s, vehicle with id 12.0 is at position (13.81, 16.65) with heading 129.68 degrees.It's x axis speed is 
-5.13 m/s and y axis speed is 3.53 m/s. It's x axis cceleration is -1.94 m/s^2 and y axis acceleration is -0.85 
m/s^2. The width of the vehicle is 2.07 m and its length is 4.51 m.
At t=-0.8 s, vehicle with id 13.0 is at position (3.50, 19.72) with heading 155.34 degrees.It's x axis speed is 
-6.64 m/s and y axis speed is 0.95 m/s. It's x axis cceleration is -1.07 m/s^2 and y axis acceleration is -1.85 
m/s^2. The width of the vehicle is 1.89 m and its length is 3.93 m.
At t=-0.64 s, vehicle with id 7.0 is at position (-47.58, -8.30) with heading 334.24 degrees.It's x axis speed is 
13.34 m/s and y axis speed is -3.03 m/s. It's x axis cceleration is 0.37 m/s^2 and y axis acceleration is -2.09 
m/s^2. The width of the vehicle is 2.08 m and its length is 4.49 m.
At t=-0.64 s, vehicle with id 8.0 is at position (35.14, -5.53) with heading 353.20 degrees.It's x axis speed is 
11.54 m/s and y axis speed is 1.37 m/s. It's x axis cceleration is 2.40 m/s^2 and y axis acceleration is -2.09 
m/s^2. The width of the vehicle is 1.94 m and its length is 5.00 m.
At t=-0.64 s, vehicle with id 9.0 is at position (22.42, 3.76) with heading 142.56 degrees.It's x axis speed is 
-5.89 m/s and y axis speed is 3.20 m/s. It's x axis cceleration is -0.09 m/s^2 and y axis acceleration is 2.57 
m/s^2. The width of the vehicle is 1.94 m and its length is 3.53 m.
At t=-0.64 s, vehicle with id 10.0 is at position (40.45, 3.84) with heading 158.95 degrees.It's x axis speed is 
-5.01 m/s and y axis speed is 0.62 m/s. It's x axis cceleration is -0.09 m/s^2 and y axis acceleration is 0.17 
m/s^2. The width of the vehicle is 2.06 m and its length is 4.94 m.
At t=-0.64 s, vehicle with id 11.0 is at position (27.84, 5.77) with heading 150.42 degrees.It's x axis speed is 
-4.47 m/s and y axis speed is 1.62 m/s. It's x axis cceleration is -0.97 m/s^2 and y axis acceleration is 1.75 
m/s^2. The width of the vehicle is 2.00 m and its length is 4.19 m.
At t=-0.64 s, vehicle with id 12.0 is at position (11.56, 18.01) with heading 135.10 degrees.It's x axis speed is 
-5.98 m/s and y axis speed is 3.14 m/s. It's x axis cceleration is -1.76 m/s^2 and y axis acceleration is -1.37 
m/s^2. The width of the vehicle is 2.07 m and its length is 4.51 m.
At t=-0.64 s

In [12]:
prompt_populator.generate_and_save_all_prompts(folder=prompts_destination)

In [13]:
# Now call the method to save the 20 longest prompts to a file
output_filename = "./test_save/top_20.json"
prompt_populator.save_top_20_longest_prompts(filename=output_filename)

print(f"The top 20 longest prompts have been saved to {output_filename}.")

The top 20 longest prompts have been saved to ./test_save/top_20.json.